In [1]:
import os  
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import matplotlib.image as mimage  
import cv2 as cv  
import random 
from PIL import Image
 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
%pwd

'd:\\Final _proj\\Driver Drawsines\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'd:\\Final _proj\\Driver Drawsines'

In [11]:
from tensorflow.keras.models import load_model 
model = load_model("drowiness_new6.h5")

## Final

In [ ]:
###

### For Yawn , no-Yawn

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

classes = ['no_yawn', 'yawn', 'Closed', 'Open']  # Class labels
IMG_SIZE = 145  # Image size for resizing
face_cascade = cv.CascadeClassifier("C:/Users/rohit/Downloads/haarcascade_frontalface_default.xml")

# Prepare the frame for prediction
def prepare_frame(frame):
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)  # Convert to grayscale for better detection
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    face_list = []
    coords = []
    
    if len(faces) > 0:
        # Select the face with the maximum area
        f = max(faces, key=lambda x: x[2] * x[3])
        x, y, w, h = f
        face = frame[y:y+h, x:x+w]  # Crop the detected face
        resized_face = cv.resize(face, (IMG_SIZE, IMG_SIZE))  # Resize to match model input
        resized_face = resized_face / 255.0  # Normalize
        face_list.append(resized_face.reshape(-1, IMG_SIZE, IMG_SIZE, 3))
        coords.append((x, y, w, h))
        
    return face_list, coords

# Start video capture
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv.flip(frame, 1)  # Flip frame horizontally
    processed_frames, coords = prepare_frame(frame)
    
    if processed_frames:
        for i, processed_frame in enumerate(processed_frames):
            prediction = model.predict(processed_frame) 
            print(prediction)
            predicted_class = np.argmax(prediction) 
            class_label = classes[predicted_class]
            x, y, w, h = coords[i]

            # Draw rectangle around the face
            cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Display predicted label on frame
            cv.putText(frame, f'{class_label}', (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Smoothen video quality by applying a Gaussian blur
    frame = cv.GaussianBlur(frame, (5, 5), 0)

    cv.imshow('Real-time Emotion Detection', frame)

    # Press 'q' to quit
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
[[0.5339012  0.15986206 0.22364037 0.08259626]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
[[0.55441564 0.16944881 0.19709249 0.07904309]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
[[0.4521452  0.13079698 0.31140834 0.10564953]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
[[0.5303667  0.16052838 0.22346936 0.08563555]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
[[0.5102811  0.15651394 0.24725401 0.08595096]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[[0.39522764 0.5094784  0.08734289 0.00795105]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
[[0.4849061  0.5083012  0.00610906 0.0006836 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
[[4.9457058e-01 4.9904898e-01 6.0036671e-03 3.7674789e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
[[4.9783722e-01 4.9717689e-01 4.6812119e-03 3.0464309e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
[[5.2321953e-01 4.6852756e-01 7.8288494e-03 4.2414822e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
[[4.9153149e-01 5.0222409e-01 5.8816043e-03 3.6273352e-0

In [23]:
cap.release()
cv.destroyAllWindows()

### For Eye open or close

In [22]:
import cv2 as cv
import numpy as np
import time
import pygame

classes = ['Closed', 'Open']  # Class labels for eyes only

# Load face and eye cascade classifiers
face_cascade = cv.CascadeClassifier("C:/Users/rohit/Downloads/haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier("C:/Users/rohit/Downloads/haarcascade_eye.xml")

# Initialize alarm
pygame.mixer.init()
pygame.mixer.music.load("research/alarm.wav")

# Track eye closure duration
closed_start_time = None
alarm_triggered = False

# Prepare the frame for eye detection
def prepare_frame(frame):
    global closed_start_time, alarm_triggered
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)  # Convert to grayscale for better detection
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    eye_status = 'Closed'
    
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        roi_gray = gray_frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
        
        # If two or more eyes detected, classify as 'Open'
        if len(eyes) >= 2:
            eye_status = 'Open'
            if alarm_triggered:
                pygame.mixer.music.stop()
                alarm_triggered = False
            closed_start_time = None
        else:
            if closed_start_time is None:
                closed_start_time = time.time()
            elif time.time() - closed_start_time >= 3:
                if not alarm_triggered:
                    pygame.mixer.music.play()
                    alarm_triggered = True
        
        # Draw a rectangle around the entire face
        cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        break  # Stop after detecting the first face to avoid multiple rectangles

    cv.putText(frame, eye_status, (20, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    return eye_status

# Start video capture
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv.flip(frame, 1)  # Flip frame horizontally
    eye_status = prepare_frame(frame)

    # Display the frame with eye detection
    cv.imshow('Real-time Eye Detection', frame)

    # Press 'q' to quit
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
